In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as font_manager
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
import warnings
warnings.filterwarnings('ignore')

In [2]:
def fbref_player_stats(url):
    
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    import re
    
    response = requests.get(url)
    print(response)
    
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("", response.text), 'lxml')
    
    player_table = soup.find_all('table')[1]
    
    player_df = pd.read_html(str(player_table))[0]
    
    level_one = player_df.columns.get_level_values(0).astype(str)
    level_two = player_df.columns.get_level_values(1).astype(str)
    new_columns = level_one + "|" + level_two
    new_columns = [x.split("|")[1] if x[0:7] == 'Unnamed' else x for x in new_columns]
    new_columns
    player_df.columns = new_columns
    
    drop_indices = player_df[player_df['Player'] == 'Player'].index
    player_df.drop(drop_indices, inplace = True)
    
    return player_df

In [3]:
defensive_actions= fbref_player_stats("https://fbref.com/en/comps/Big5/defense/players/Big-5-European-Leagues-Stats")
num_columns=['90s','Tackles|Tkl','Tackles|TklW','Tackles|Def 3rd','Tackles|Mid 3rd','Tackles|Att 3rd','Vs Dribbles|Tkl',
             'Vs Dribbles|Att','Vs Dribbles|Tkl%','Vs Dribbles|Past','Pressures|Press','Pressures|Succ','Pressures|%',
             'Pressures|Def 3rd','Pressures|Mid 3rd','Pressures|Att 3rd','Blocks|Blocks','Blocks|Sh','Blocks|ShSv','Blocks|Pass',
             'Int','Tkl+Int','Clr','Err']
defensive_actions[num_columns] = defensive_actions[num_columns].apply(pd.to_numeric)
defensive_actions.drop('Matches', axis=1, inplace=True)
defensive_actions.head()

<Response [200]>


,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Tackles|Tkl,...,Pressures|Mid 3rd,Pressures|Att 3rd,Blocks|Blocks,Blocks|Sh,Blocks|ShSv,Blocks|Pass,Int,Tkl+Int,Clr,Err
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,30-066,1990,1.2,0.0,...,2.0,1.0,3.0,0.0,0.0,3.0,0,0,4.0,0.0
1,2,Issah Abbas,gh GHA,FW,Mainz 05,de Bundesliga,22-038,1998,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,9.0,...,28.0,4.0,11.0,5.0,0.0,6.0,10,19,47.0,0.0
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,6.0,...,32.0,12.0,5.0,1.0,0.0,4.0,4,10,5.0,1.0
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,50.0,...,135.0,17.0,9.0,2.0,0.0,7.0,3,53,2.0,0.0


In [4]:
def player_filter_fbref(dataframe, 
                        mins_filter = 3.33, 
                        position_filter = [], 
                        nation_filter = [], 
                        squad_filter = [], 
                        comp_filter = []):
    
    filtered_df = dataframe.copy()
    
    if len(str(mins_filter)) > 0:
        filtered_df = filtered_df[filtered_df['90s'] > mins_filter]
    #if len(str(age_min)) > 0:
     #   filtered_df = filtered_df[filtered_df['Age'] > age_min]
    #if len(str(age_max)) > 0:
     #   filtered_df = filtered_df[filtered_df['Age'] < age_max]
    if len(position_filter) > 0:
        filtered_df = filtered_df[(filtered_df['Pos'].isin(position_filter))]
    if len(nation_filter) > 0:
        filtered_df = filtered_df[(filtered_df['Nation'].isin(nation_filter))]
    if len(squad_filter) > 0:
        filtered_df = filtered_df[(filtered_df['Squad'].isin(squad_filter))]
    if len(comp_filter) > 0:
        filtered_df = filtered_df[(filtered_df['Comp'].isin(comp_filter))]
        
    return filtered_df

In [5]:
defensive_filtered = player_filter_fbref(defensive_actions,
                                       #mins_filter = 2,
                        
                                       position_filter = [],
                                       nation_filter = [],
                                       squad_filter = [],
                                       comp_filter = [])
defensive_filtered.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Tackles|Tkl,...,Pressures|Mid 3rd,Pressures|Att 3rd,Blocks|Blocks,Blocks|Sh,Blocks|ShSv,Blocks|Pass,Int,Tkl+Int,Clr,Err
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,9.0,...,28.0,4.0,11.0,5.0,0.0,6.0,10,19,47.0,0.0
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,6.0,...,32.0,12.0,5.0,1.0,0.0,4.0,4,10,5.0,1.0
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,50.0,...,135.0,17.0,9.0,2.0,0.0,7.0,3,53,2.0,0.0
6,7,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34-111,1986,4.7,5.0,...,25.0,8.0,13.0,2.0,0.0,11.0,5,10,12.0,0.0
10,11,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32-267,1988,6.0,15.0,...,29.0,11.0,16.0,3.0,0.0,13.0,9,24,28.0,0.0


In [6]:
defense_cols =  ['Tackles|Tkl','Tackles|TklW','Tackles|Def 3rd','Tackles|Mid 3rd','Tackles|Att 3rd','Vs Dribbles|Tkl',
             'Vs Dribbles|Att','Vs Dribbles|Tkl%','Vs Dribbles|Past','Pressures|Press','Pressures|Succ','Pressures|%',
             'Pressures|Def 3rd','Pressures|Mid 3rd','Pressures|Att 3rd','Blocks|Blocks','Blocks|Sh','Blocks|ShSv','Blocks|Pass',
             'Int','Tkl+Int','Clr','Err']
for col in defense_cols:
    defensive_filtered[col + '|90'] = defensive_filtered[col] / defensive_filtered['90s']

defensive_filtered.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Tackles|Tkl,...,Pressures|Mid 3rd|90,Pressures|Att 3rd|90,Blocks|Blocks|90,Blocks|Sh|90,Blocks|ShSv|90,Blocks|Pass|90,Int|90,Tkl+Int|90,Clr|90,Err|90
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,9.0,...,3.500000,0.500000,1.375000,0.625000,0.0,0.750000,1.250000,2.375000,5.875000,0.000000
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,6.0,...,7.272727,2.727273,1.136364,0.227273,0.0,0.909091,0.909091,2.272727,1.136364,0.227273
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,50.0,...,16.666667,2.098765,1.111111,0.246914,0.0,0.864198,0.370370,6.543210,0.246914,0.000000
6,7,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34-111,1986,4.7,5.0,...,5.319149,1.702128,2.765957,0.425532,0.0,2.340426,1.063830,2.127660,2.553191,0.000000
10,11,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32-267,1988,6.0,15.0,...,4.833333,1.833333,2.666667,0.500000,0.0,2.166667,1.500000,4.000000,4.666667,0.000000


In [28]:
defensive_filtered.to_csv(r'C:\Users\nikhi\Desktop\Projects\defense2.csv')

In [8]:
passing = fbref_player_stats("https://fbref.com/en/comps/Big5/passing/players/Big-5-European-Leagues-Stats")
num_columns = ['90s',
               'Total|Cmp', 'Total|Att', 'Total|Cmp%', 'Total|TotDist', 'Total|PrgDist', 
               'Short|Cmp', 'Short|Att', 'Short|Cmp%', 
               'Medium|Cmp','Medium|Att', 'Medium|Cmp%', 
               'Long|Cmp', 'Long|Att', 'Long|Cmp%', 
               'Ast', 'xA', 'A-xA', 'KP', 
               '1/3', 'PPA', 'CrsPA', 'Prog']
passing[num_columns] = passing[num_columns].apply(pd.to_numeric)
passing.drop('Matches', axis=1, inplace=True)
passing.head()

<Response [200]>


,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Total|Cmp,...,Long|Att,Long|Cmp%,Ast,xA,A-xA,KP,1/3,PPA,CrsPA,Prog
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,30-066,1990,1.2,67.0,...,6.0,66.7,0,0.1,-0.1,1.0,6.0,5.0,1.0,9.0
1,2,Issah Abbas,gh GHA,FW,Mainz 05,de Bundesliga,22-038,1998,0.0,2.0,...,1.0,100.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,356.0,...,94.0,75.5,0,0.0,0.0,0.0,16.0,0.0,0.0,17.0
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,158.0,...,54.0,61.1,0,0.4,-0.4,5.0,25.0,3.0,2.0,11.0
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,310.0,...,66.0,75.8,0,0.2,-0.2,2.0,26.0,3.0,2.0,26.0


In [9]:
passing_filtered = player_filter_fbref(passing,
                                       mins_filter = 3.33,
                                
                                       position_filter = [],
                                       nation_filter = [],
                                       squad_filter = [],
                                       comp_filter = [])
passing_filtered.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Total|Cmp,...,Long|Att,Long|Cmp%,Ast,xA,A-xA,KP,1/3,PPA,CrsPA,Prog
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,356.0,...,94.0,75.5,0,0.0,0.0,0.0,16.0,0.0,0.0,17.0
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,158.0,...,54.0,61.1,0,0.4,-0.4,5.0,25.0,3.0,2.0,11.0
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,310.0,...,66.0,75.8,0,0.2,-0.2,2.0,26.0,3.0,2.0,26.0
6,7,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34-111,1986,4.7,241.0,...,71.0,62.0,0,0.4,-0.4,2.0,21.0,3.0,2.0,20.0
10,11,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32-267,1988,6.0,347.0,...,74.0,66.2,1,0.2,0.8,2.0,18.0,4.0,4.0,19.0


In [10]:
pass_cols =  ['Total|Cmp', 'Total|Att', 'Total|Cmp%', 
                'Total|TotDist', 'Total|PrgDist', 
                'Short|Cmp', 'Short|Att', 'Short|Cmp%', 
                'Medium|Cmp','Medium|Att', 'Medium|Cmp%', 
                'Long|Cmp', 'Long|Att', 'Long|Cmp%', 
                'Ast', 'xA', 'A-xA', 'KP', 
                '1/3', 'PPA', 'CrsPA', 'Prog']

for col in pass_cols:
    passing_filtered[col + '|90'] = passing_filtered[col] / passing_filtered['90s']

passing_filtered.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Total|Cmp,...,Long|Att|90,Long|Cmp%|90,Ast|90,xA|90,A-xA|90,KP|90,1/3|90,PPA|90,CrsPA|90,Prog|90
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,356.0,...,11.750000,9.437500,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.125000
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,158.0,...,12.272727,13.886364,0.000000,0.090909,-0.090909,1.136364,5.681818,0.681818,0.454545,2.500000
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,310.0,...,8.148148,9.358025,0.000000,0.024691,-0.024691,0.246914,3.209877,0.370370,0.246914,3.209877
6,7,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34-111,1986,4.7,241.0,...,15.106383,13.191489,0.000000,0.085106,-0.085106,0.425532,4.468085,0.638298,0.425532,4.255319
10,11,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32-267,1988,6.0,347.0,...,12.333333,11.033333,0.166667,0.033333,0.133333,0.333333,3.000000,0.666667,0.666667,3.166667


In [11]:
passing_filtered.to_csv(r'C:\Users\nikhi\Desktop\Projects\Football analysis\passing2.csv')

In [12]:
misc=fbref_player_stats("https://fbref.com/en/comps/Big5/2019-2020/misc/players/2019-2020-Big-5-European-Leagues-Stats")
num_columns = ['90s',
               'Performance|CrdY','Performance|CrdR','Performance|2CrdY','Performance|Fls','Performance|Fld','Performance|Off',
              'Performance|Crs','Performance|Int','Performance|TklW','Performance|PKwon','Performance|PKcon','Performance|OG','Performance|Recov',
              'Aerial Duels|Won','Aerial Duels|Lost','Aerial Duels|Won%']
misc[num_columns] = misc[num_columns].apply(pd.to_numeric)
misc.drop('Matches', axis=1, inplace=True)
misc.head()

<Response [200]>


,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Performance|CrdY,...,Performance|Crs,Performance|Int,Performance|TklW,Performance|PKwon,Performance|PKcon,Performance|OG,Performance|Recov,Aerial Duels|Won,Aerial Duels|Lost,Aerial Duels|Won%
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,28,1990,27.9,0,...,44,23,32,0.0,0.0,1,243.0,12.0,17.0,41.4
1,2,Max Aarons,eng ENG,DF,Norwich City,eng Premier League,19,2000,36.0,7,...,54,20,36,1.0,0.0,0,281.0,23.0,42.0,35.4
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,31,1987,28.0,0,...,1,24,37,0.0,1.0,0,226.0,86.0,42.0,67.2
3,4,Suleiman Abdullahi,ng NGA,"FW,MF",Union Berlin,de Bundesliga,22,1996,1.7,1,...,7,1,1,0.0,0.0,0,15.0,6.0,11.0,35.3
4,5,Mehdi Abeid,dz ALG,MF,Nantes,fr Ligue 1,26,1992,20.2,5,...,18,17,41,0.0,1.0,0,194.0,14.0,19.0,42.4


In [13]:
misc_filtered = player_filter_fbref(misc,
                                       mins_filter = 3.33,
                                
                                       position_filter = [],
                                       nation_filter = [],
                                       squad_filter = [],
                                       comp_filter = [])
misc_filtered.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Performance|CrdY,...,Performance|Crs,Performance|Int,Performance|TklW,Performance|PKwon,Performance|PKcon,Performance|OG,Performance|Recov,Aerial Duels|Won,Aerial Duels|Lost,Aerial Duels|Won%
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,28,1990,27.9,0,...,44,23,32,0.0,0.0,1,243.0,12.0,17.0,41.4
1,2,Max Aarons,eng ENG,DF,Norwich City,eng Premier League,19,2000,36.0,7,...,54,20,36,1.0,0.0,0,281.0,23.0,42.0,35.4
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,31,1987,28.0,0,...,1,24,37,0.0,1.0,0,226.0,86.0,42.0,67.2
4,5,Mehdi Abeid,dz ALG,MF,Nantes,fr Ligue 1,26,1992,20.2,5,...,18,17,41,0.0,1.0,0,194.0,14.0,19.0,42.4
5,6,Charles Abi,fr FRA,FW,Saint-Étienne,fr Ligue 1,19,2000,6.9,2,...,4,4,8,1.0,0.0,0,25.0,27.0,39.0,40.9


In [14]:
misc_cols =  ['Performance|CrdY','Performance|CrdR','Performance|2CrdY','Performance|Fls','Performance|Fld','Performance|Off',
              'Performance|Crs','Performance|Int','Performance|TklW','Performance|PKwon','Performance|PKcon','Performance|OG','Performance|Recov',
              'Aerial Duels|Won','Aerial Duels|Lost','Aerial Duels|Won%']

for col in misc_cols:
    misc_filtered[col + '|90'] = misc_filtered[col] / misc_filtered['90s']

misc_filtered.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Performance|CrdY,...,Performance|Crs|90,Performance|Int|90,Performance|TklW|90,Performance|PKwon|90,Performance|PKcon|90,Performance|OG|90,Performance|Recov|90,Aerial Duels|Won|90,Aerial Duels|Lost|90,Aerial Duels|Won%|90
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,28,1990,27.9,0,...,1.577061,0.824373,1.146953,0.000000,0.000000,0.035842,8.709677,0.430108,0.609319,1.483871
1,2,Max Aarons,eng ENG,DF,Norwich City,eng Premier League,19,2000,36.0,7,...,1.500000,0.555556,1.000000,0.027778,0.000000,0.000000,7.805556,0.638889,1.166667,0.983333
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,31,1987,28.0,0,...,0.035714,0.857143,1.321429,0.000000,0.035714,0.000000,8.071429,3.071429,1.500000,2.400000
4,5,Mehdi Abeid,dz ALG,MF,Nantes,fr Ligue 1,26,1992,20.2,5,...,0.891089,0.841584,2.029703,0.000000,0.049505,0.000000,9.603960,0.693069,0.940594,2.099010
5,6,Charles Abi,fr FRA,FW,Saint-Étienne,fr Ligue 1,19,2000,6.9,2,...,0.579710,0.579710,1.159420,0.144928,0.000000,0.000000,3.623188,3.913043,5.652174,5.927536


In [15]:
misc_filtered.to_csv(r'C:\Users\nikhi\Desktop\Projects\Football analysis\misc2.csv')

In [16]:
shots=fbref_player_stats("https://fbref.com/en/comps/Big5/shooting/players/Big-5-European-Leagues-Stats")
num_columns=['90s','Standard|Gls','Standard|Sh','Standard|SoT','Standard|SoT%','Standard|Sh/90','Standard|SoT/90','Standard|G/Sh','Standard|G/SoT','Standard|FK','Standard|PK','Standard|PKatt',
 'Expected|xG','Expected|npxG','Expected|npxG/Sh','Expected|G-xG','Expected|np:G-xG']            
shots[num_columns] = shots[num_columns].apply(pd.to_numeric)
shots.drop('Matches', axis=1, inplace=True)
shots.head()

<Response [200]>


,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Standard|Gls,...,Standard|G/Sh,Standard|G/SoT,Standard|FK,Standard|PK,Standard|PKatt,Expected|xG,Expected|npxG,Expected|npxG/Sh,Expected|G-xG,Expected|np:G-xG
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,30-066,1990,1.2,0,...,0.0,0.0,1.0,0,0,0.1,0.1,0.04,-0.1,-0.1
1,2,Issah Abbas,gh GHA,FW,Mainz 05,de Bundesliga,22-038,1998,0.0,0,...,NaN,NaN,0.0,0,0,0.0,0.0,NaN,0.0,0.0
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,1,...,0.5,1.0,0.0,0,0,0.3,0.3,0.16,0.7,0.7
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,0,...,0.0,NaN,0.0,0,0,0.3,0.3,0.05,-0.3,-0.3
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,0,...,0.0,0.0,0.0,0,0,0.1,0.1,0.02,-0.1,-0.1


In [17]:
shots_filtered = player_filter_fbref(shots,
                                       mins_filter = 3.33,
                                
                                       position_filter = [],
                                       nation_filter = [],
                                       squad_filter = [],
                                       comp_filter = [])
shots_filtered.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Standard|Gls,...,Standard|G/Sh,Standard|G/SoT,Standard|FK,Standard|PK,Standard|PKatt,Expected|xG,Expected|npxG,Expected|npxG/Sh,Expected|G-xG,Expected|np:G-xG
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,1,...,0.5,1.0,0.0,0,0,0.3,0.3,0.16,0.7,0.7
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,0,...,0.0,NaN,0.0,0,0,0.3,0.3,0.05,-0.3,-0.3
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,0,...,0.0,0.0,0.0,0,0,0.1,0.1,0.02,-0.1,-0.1
6,7,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34-111,1986,4.7,0,...,0.0,0.0,0.0,0,0,0.3,0.3,0.08,-0.3,-0.3
10,11,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32-267,1988,6.0,0,...,0.0,NaN,0.0,0,0,0.0,0.0,0.04,0.0,0.0


In [18]:
shots_cols =  ['90s','Standard|Gls','Standard|Sh','Standard|SoT','Standard|SoT%','Standard|G/Sh','Standard|G/SoT','Standard|FK','Standard|PK','Standard|PKatt',
 'Expected|xG','Expected|npxG','Expected|npxG/Sh','Expected|G-xG','Expected|np:G-xG']
#shots_cols= ['90s','Standard|Gls','Standard|Sh','Standard|SoT','Standard|SoT%','Standard|Sh/90','Standard|SoT/90','Standard|G/Sh','Standard|G/SoT','Standard|FK','Standard|PK','Standard|PKatt',
 #'Expected|xG','Expected|npxG','Expected|npxG/Sh','Expected|G-xG','Expected|np:G-xG']

for col in shots_cols:
    shots_filtered[col + '|90'] = shots_filtered[col] / shots_filtered['90s']

shots_filtered.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Standard|Gls,...,Standard|G/Sh|90,Standard|G/SoT|90,Standard|FK|90,Standard|PK|90,Standard|PKatt|90,Expected|xG|90,Expected|npxG|90,Expected|npxG/Sh|90,Expected|G-xG|90,Expected|np:G-xG|90
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,1,...,0.0625,0.125,0.0,0.0,0.0,0.037500,0.037500,0.020000,0.087500,0.087500
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,0,...,0.0000,NaN,0.0,0.0,0.0,0.068182,0.068182,0.011364,-0.068182,-0.068182
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,0,...,0.0000,0.000,0.0,0.0,0.0,0.012346,0.012346,0.002469,-0.012346,-0.012346
6,7,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34-111,1986,4.7,0,...,0.0000,0.000,0.0,0.0,0.0,0.063830,0.063830,0.017021,-0.063830,-0.063830
10,11,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32-267,1988,6.0,0,...,0.0000,NaN,0.0,0.0,0.0,0.000000,0.000000,0.006667,0.000000,0.000000


In [19]:
shots_filtered.to_csv(r'C:\Users\nikhi\Desktop\Projects\Football analysis\shots2.csv')

In [20]:
gca=fbref_player_stats("https://fbref.com/en/comps/Big5/gca/players/Big-5-European-Leagues-Stats")
num_columns=['90s','SCA|SCA','SCA|SCA90','SCA Types|PassLive','SCA Types|PassDead','SCA Types|Drib','SCA Types|Sh','SCA Types|Fld',
            'SCA Types|Def','GCA|GCA','GCA|GCA90','GCA Types|PassLive','GCA Types|PassDead','GCA Types|Drib','GCA Types|Sh',
            'GCA Types|Fld','GCA Types|Def','GCA Types|OG']
gca[num_columns] = gca[num_columns].apply(pd.to_numeric)
gca.drop('Matches', axis=1, inplace=True)
gca.head()

<Response [200]>


,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,SCA|SCA,...,SCA Types|Def,GCA|GCA,GCA|GCA90,GCA Types|PassLive,GCA Types|PassDead,GCA Types|Drib,GCA Types|Sh,GCA Types|Fld,GCA Types|Def,GCA Types|OG
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,30-066,1990,1.2,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Issah Abbas,gh GHA,FW,Mainz 05,de Bundesliga,22-038,1998,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,7.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
gca_filtered = player_filter_fbref(gca,
                                       mins_filter = 3.33,
                                
                                       position_filter = [],
                                       nation_filter = [],
                                       squad_filter = [],
                                       comp_filter = [])
gca_filtered.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,SCA|SCA,...,SCA Types|Def,GCA|GCA,GCA|GCA90,GCA Types|PassLive,GCA Types|PassDead,GCA Types|Drib,GCA Types|Sh,GCA Types|Fld,GCA Types|Def,GCA Types|OG
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,9.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,7.0,...,1.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34-111,1986,4.7,4.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,11,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32-267,1988,6.0,3.0,...,0.0,1.0,0.17,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
gca_cols =  ['90s','SCA|SCA','SCA Types|PassLive','SCA Types|PassDead','SCA Types|Drib','SCA Types|Sh','SCA Types|Fld',
            'SCA Types|Def','GCA|GCA','GCA Types|PassLive','GCA Types|PassDead','GCA Types|Drib','GCA Types|Sh',
            'GCA Types|Fld','GCA Types|Def','GCA Types|OG']
for col in gca_cols:
    gca_filtered[col + '|90'] = gca_filtered[col] / gca_filtered['90s']

gca_filtered.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,SCA|SCA,...,SCA Types|Fld|90,SCA Types|Def|90,GCA|GCA|90,GCA Types|PassLive|90,GCA Types|PassDead|90,GCA Types|Drib|90,GCA Types|Sh|90,GCA Types|Fld|90,GCA Types|Def|90,GCA Types|OG|90
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,9.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,7.0,...,0.123457,0.123457,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
6,7,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34-111,1986,4.7,4.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
10,11,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32-267,1988,6.0,3.0,...,0.000000,0.000000,0.166667,0.166667,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
gca_filtered.to_csv(r'C:\Users\nikhi\Desktop\Projects\gca.csv')

In [24]:
poss=fbref_player_stats("https://fbref.com/en/comps/Big5/possession/players/Big-5-European-Leagues-Stats")
num_columns=['90s','Touches|Touches','Touches|Def Pen','Touches|Def 3rd','Touches|Mid 3rd','Touches|Att 3rd','Touches|Att Pen',
            'Touches|Live','Dribbles|Succ','Dribbles|Att','Dribbles|Succ%','Dribbles|#Pl','Dribbles|Megs','Carries|Carries',
            'Carries|TotDist','Carries|PrgDist','Receiving|Targ','Receiving|Rec','Receiving|Rec%','Miscon','Dispos']
poss[num_columns]=poss[num_columns].apply(pd.to_numeric)
poss.drop('Matches',axis=1,inplace=True)
poss.head()

<Response [200]>


,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Touches|Touches,...,Dribbles|#Pl,Dribbles|Megs,Carries|Carries,Carries|TotDist,Carries|PrgDist,Receiving|Targ,Receiving|Rec,Receiving|Rec%,Miscon,Dispos
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,30-066,1990,1.2,88.0,...,0.0,0.0,56.0,329.0,204.0,69.0,63.0,91.3,0.0,0.0
1,2,Issah Abbas,gh GHA,FW,Mainz 05,de Bundesliga,22-038,1998,0.0,3.0,...,0.0,0.0,2.0,35.0,29.0,3.0,3.0,100.0,0.0,0.0
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,481.0,...,2.0,0.0,298.0,1658.0,962.0,337.0,328.0,97.3,4.0,5.0
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,231.0,...,5.0,0.0,144.0,970.0,522.0,169.0,152.0,89.9,2.0,1.0
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,462.0,...,3.0,0.0,308.0,1443.0,795.0,307.0,290.0,94.5,6.0,9.0


In [25]:
poss_filtered = player_filter_fbref(poss,
                                       mins_filter = 3.33,
                                
                                       position_filter = [],
                                       nation_filter = [],
                                       squad_filter = [],
                                       comp_filter = [])
poss_filtered.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Touches|Touches,...,Dribbles|#Pl,Dribbles|Megs,Carries|Carries,Carries|TotDist,Carries|PrgDist,Receiving|Targ,Receiving|Rec,Receiving|Rec%,Miscon,Dispos
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,481.0,...,2.0,0.0,298.0,1658.0,962.0,337.0,328.0,97.3,4.0,5.0
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,231.0,...,5.0,0.0,144.0,970.0,522.0,169.0,152.0,89.9,2.0,1.0
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,462.0,...,3.0,0.0,308.0,1443.0,795.0,307.0,290.0,94.5,6.0,9.0
6,7,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34-111,1986,4.7,341.0,...,2.0,0.0,162.0,953.0,568.0,207.0,201.0,97.1,0.0,0.0
10,11,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32-267,1988,6.0,480.0,...,1.0,0.0,271.0,1663.0,778.0,305.0,299.0,98.0,5.0,1.0


In [26]:
poss_cols =  ['Touches|Touches','Touches|Def Pen','Touches|Def 3rd','Touches|Mid 3rd','Touches|Att 3rd','Touches|Att Pen',
            'Touches|Live','Dribbles|Succ','Dribbles|Att','Dribbles|Succ%','Dribbles|#Pl','Dribbles|Megs','Carries|Carries',
            'Carries|TotDist','Carries|PrgDist','Receiving|Targ','Receiving|Rec','Receiving|Rec%','Miscon','Dispos']
for col in poss_cols:
    poss_filtered[col + '|90'] = poss_filtered[col] / poss_filtered['90s']

poss_filtered.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Touches|Touches,...,Dribbles|#Pl|90,Dribbles|Megs|90,Carries|Carries|90,Carries|TotDist|90,Carries|PrgDist|90,Receiving|Targ|90,Receiving|Rec|90,Receiving|Rec%|90,Miscon|90,Dispos|90
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-036,1987,8.0,481.0,...,0.250000,0.0,37.250000,207.250000,120.250000,42.125000,41.000000,12.162500,0.500000,0.625000
3,4,Mehdi Abeid,dz ALG,"MF,DF",Nantes,fr Ligue 1,28-089,1992,4.4,231.0,...,1.136364,0.0,32.727273,220.454545,118.636364,38.409091,34.545455,20.431818,0.454545,0.227273
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-276,1993,8.1,462.0,...,0.370370,0.0,38.024691,178.148148,98.148148,37.901235,35.802469,11.666667,0.740741,1.111111
6,7,David Abraham,ar ARG,DF,Eint Frankfurt,de Bundesliga,34-111,1986,4.7,341.0,...,0.425532,0.0,34.468085,202.765957,120.851064,44.042553,42.765957,20.659574,0.000000,0.000000
10,11,Francesco Acerbi,it ITA,DF,Lazio,it Serie A,32-267,1988,6.0,480.0,...,0.166667,0.0,45.166667,277.166667,129.666667,50.833333,49.833333,16.333333,0.833333,0.166667


In [27]:
poss_filtered.to_csv(r'C:\Users\nikhi\Desktop\Projects\poss.csv')